In [1]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models

     |████████████████████████████████| 1.7 MB 9.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=47e7d08169f113326bfda3904ccef74c254f16315324c32976cecc02539f5e52
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/local/lib/python3.7/dist-packa

In [ ]:
import nltk
nltk.download('all')

In [ ]:
import pandas as pd
from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim
import gensim.corpora as corpora
import re
import os
import seaborn as sns
from gensim.models.coherencemodel import CoherenceModel
import numpy as np
import glob

In [5]:
#create functions
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.ADV
    

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags if get_wordnet_pos(t[1]) == wordnet.NOUN]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return final

def make_corpus(data_words):
    corpus = []
    for text in data_words:
        id2word = corpora.Dictionary(data_words)
        new = id2word.doc2bow(text)
        corpus.append(new)
    return corpus


def LDA(lst, file_name, ntopics):
    file_name = file_name
    lemmatized_texts = lst
    data_words = gen_words(lemmatized_texts)
    corpus = make_corpus(data_words)
    id2word = corpora.Dictionary(data_words)
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=ntopics,
                                               random_state=0,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha="auto")
    vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds")
    pyLDAvis.save_html(vis, file_name+ ".html")
    print("[0]: lda_model\n[1]: corpus\n[2]: id2word")
    return (lda_model, corpus, id2word)

def LDAnoVis(lst, ntopics):
    lemmatized_texts = lst
    data_words = gen_words(lemmatized_texts)
    corpus = make_corpus(data_words)
    id2word = corpora.Dictionary(data_words)
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=ntopics,
                                               random_state=0,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha="auto")
    return (lda_model, corpus, id2word)


def CoherenceLDA(data, iteration):
    Colst = []
    times = 0
    for i in range(1, iteration + 1):
        res = LDAnoVis(data, i)
        cm = CoherenceModel(
            model=res[0],
            corpus=res[1],
            dictionary=res[2],
            coherence='u_mass', # Coherenceの算出方法を指定。 (デフォルトは'c_v')
            topn=20 # 各トピックの上位何単語から算出するか指定(デフォルト20)
            )
        Colst.append(cm.get_coherence())
        times = times + 1
        print(str(times) + "/" + str(iteration))
    
    df = pd.DataFrame()
    df["N_of_Topic"] = list(range(1, iteration + 1))
    df["Coherence"] = Colst
    df["min_max_Coh"] = (df["Coherence"] - df["Coherence"].min()) / (df["Coherence"].max() - df["Coherence"].min())
    return df

def PerplexityLDA(data, iteration):
    Perlst = []
    times = 0
    for i in range(1, iteration + 1):
        res = LDAnoVis(data, i)
        per = np.exp(-res[0].log_perplexity(res[1]))
        Perlst.append(per)
        times = times + 1
        print(str(times) + "/" + str(iteration))
    
    df = pd.DataFrame()
    df["N_of_Topic"] = list(range(1, iteration + 1))
    df["Perplexity"] = Perlst
    df["min_max_Per"] = (df["Perplexity"] - df["Perplexity"].min()) / (df["Perplexity"].max() - df["Perplexity"].min())
    return df

def gen_dfwords(lda, ntopics):
    twords = []
    rows = []
    for i in range(0,ntopics):
        rows.append("Topic" + str(i+1))
        twords.append([ word.split('"')[1] for word in lda.print_topic(i).split('+') ])
   
    dfwords = pd.DataFrame(twords, index=rows)
    return dfwords
    

def gen_dfByLDA(lda, tcorpus, ntopics):
  #get the value of topics in each documents
    topics = []
    for c in tcorpus:
        topic = [0] * ntopics
        for (tpc, prob) in lda.get_document_topics(c):
            topic[tpc] = prob
        topics.append(topic)
  #get the highest value of topic in each documents
    clusters = []
    for x in topics:
        clusters.append(x.index(max(x)))
    #cluster means the highest value of topic in each documents
    return [topics, clusters]

In [6]:
#create a function so we can just input a textdata then we get all we want
#This function need a list of text input
def brieflyLDA_CohPer(folderpath, iteration):
  globpath = glob.glob(folderpath + "/*.txt")
  text_lst = []
  for txt in globpath:
    with open(txt) as f:
      text_lst.append(f.read())

  text_lst_cleaned = [clean_text(i) for i in text_lst]
  df_Coh = CoherenceLDA(text_lst_cleaned, iteration)
  df_Per = PerplexityLDA(text_lst_cleaned, iteration)
  df = pd.merge(df_Coh, df_Per, left_index=True, right_index=True)
  df.to_excel(folderpath + ".xlsx", index=False)

In [7]:
def brieflyLDA(folderpath, num_of_topics):
  globpath = glob.glob(folderpath + "/*.txt")
  text_lst = []
  for txt in globpath:
    with open(txt) as f:
      text_lst.append(f.read())

  text_lst_cleaned = [clean_text(i) for i in text_lst]
  file_name = folderpath.replace("/content/", "")
  LDAmodel = LDA(text_lst_cleaned, file_name, num_of_topics)
  article_name_lst = [(re.search("((?:[^/]*/)*)(.*)", path)).groups()[1] for path in globpath]

  Topic_of_doc = gen_dfByLDA(LDAmodel[0], LDAmodel[1], num_of_topics)
  df = pd.DataFrame({"Article": article_name_lst, "Topic": Topic_of_doc[1]})
  df.to_excel(file_name + ".xlsx", index=False)
  return df

In [ ]:
"/content/before2011".replace("/content/", "") + ".xlsx"

'before2011.xlsx'

In [ ]:
!unzip "/content/2012to2015.zip"
!unzip "/content/2016to2019.zip"
!unzip "/content/2020to2021.zip"
!unzip "/content/before2011.zip"

In [ ]:
!unzip "/content/2016to2019.zip"
!unzip "/content/2020to2021.zip"

In [ ]:
brieflyLDA_CohPer("/content/2012to2015", 20)
brieflyLDA_CohPer("/content/2016to2019", 20)
brieflyLDA_CohPer("/content/2020to2021", 20)
brieflyLDA_CohPer("/content/before2011", 20)

In [ ]:
brieflyLDA_CohPer("/content/2020to2021", 20)

In [ ]:
LDAbefore2011 = brieflyLDA("/content/before2011", 12)
LDA2012to2015 = brieflyLDA("/content/2012to2015", 5)
LDA2016to2019 = brieflyLDA("/content/2016to2019", 8)
LDA2020to2021 = brieflyLDA("/content/2020to2021", 3)

In [ ]:
LDA2020to2021 = brieflyLDA("/content/2020to2021", 3)